<a href="https://colab.research.google.com/github/Harsha774/UTD-Masters-Work-Portfolio/blob/master/Spark_Transformations_n_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [3]:
!pip install -U pyarrow

     |████████████████████████████████| 23.6 MB 104 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 3.0.0
    Uninstalling pyarrow-3.0.0:
      Successfully uninstalled pyarrow-3.0.0


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.executer.memory","4g")
spark.conf.set("spark.driver.fraction","0.9")

In [7]:
import sys,tempfile,urllib
from pyspark.sql.functions import *

In [8]:
BASE_DIR='/tmp'
corona_data_file=os.path.join(BASE_DIR,'corona_data.csv')

In [9]:
import urllib.request
corona_data=urllib.request.urlretrieve("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv",corona_data_file)

In [10]:
corona_df=spark.read.option("infer_schema","true").csv("/tmp/corona_data.csv",header=True)

In [11]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [12]:
BASE_DIR='/tmp'
twitter_data_file=os.path.join(BASE_DIR,'tweets.csv')

In [13]:
corona_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Death: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [14]:
import urllib.request
corona_data=urllib.request.urlretrieve("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv",twitter_data_file)

In [15]:
ls /tmp

blockmgr-f134b599-aabf-4a9f-91b2-6fdf65179e91/
corona_data.csv
dap_multiplexer.45f6f66e1d50.root.log.INFO.20210923-151939.56
dap_multiplexer.INFO@
debugger_puwldhbm5=
hsperfdata_root/
initgoogle_syslog_dir.0/
liblz4-java-2579035530076325752.so
liblz4-java-2579035530076325752.so.lck
pyright-102-BP91TmKfkFkM/
pyright-102-FacEh9rnhHHa/
pyright-237-mjASPyQqmDk4/
pyright-237-YJWOJNLmUubq/
pyright-263-dBxtCaap11lD/
pyright-263-NNeQslP7xFeG/
python-languageserver-cancellation/
spark-578523bb-f9d0-45c3-b9b4-4243cecdaac2/
spark-b00ddc9c-9862-4210-aae7-734956a8cd88/
tweets.csv


In [16]:
!head /tmp/tweets.csv

,geo,text,user,location,entities,sentiment,country
0,,What is God saying to us about #coronavirus ?,petodinice,Lagos,"[('about #', 'CARDINAL')]","{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}",Nigeria
1,,"BREAKING: ""this is disappointing, but i took the test"". Arsenal's Mikel Arteta tests positive for #coronavirus,  -… ",JerryfranksonJF,"Abuja, Nigeria","[(""Arsenal's Mikel Arteta"", 'ORG'), '']","{'neg': 0.1, 'neu': 0.667, 'pos': 0.233, 'compound': 0.5859}",Nigeria
2,, #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…,cek422,"Pennsylvania, USA",[],"{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}",USA
3,, Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…,InfectiousDz,NYC,"[('World', 'ORG'), ('this weekend', 'DATE'), '']","{'neg': 0.085, 'neu': 0.798, 'pos': 0.117, 'compound': 0.1779}",USA
4,, The #coronavirus pande

In [17]:
twitter_df=spark.read.option("infer_schema","true").csv("/tmp/tweets.csv",header=True)

In [18]:
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [19]:
twitter_df.count()

1000

In [20]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [21]:
twitter_df.filter("country='USA'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [22]:
twitter_df.filter("country='USA'").count()

364

In [23]:
twitter_df.filter("country='USA' and location like '%New%'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [24]:
tdf=twitter_df.filter("country='USA'")

In [25]:
tdf.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [26]:
tdf.explain()

== Physical Plan ==
*(1) Filter (isnotnull(country#117) AND (country#117 = USA))
+- FileScan csv [_c0#110,geo#111,text#112,user#113,location#114,entities#115,sentiment#116,country#117] Batched: false, DataFilters: [isnotnull(country#117), (country#117 = USA)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/tweets.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,USA)], ReadSchema: struct<_c0:string,geo:string,text:string,user:string,location:string,entities:string,sentiment:st...




In [27]:
tdf.count()

364

In [28]:
tdf=twitter_df.filter("country='USA' and location like '%New%'")

In [29]:
tdf.explain()

== Physical Plan ==
*(1) Filter (((isnotnull(country#117) AND isnotnull(location#114)) AND (country#117 = USA)) AND Contains(location#114, New))
+- FileScan csv [_c0#110,geo#111,text#112,user#113,location#114,entities#115,sentiment#116,country#117] Batched: false, DataFilters: [isnotnull(country#117), isnotnull(location#114), (country#117 = USA), Contains(location#114, New)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/tweets.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), IsNotNull(location), EqualTo(country,USA), StringContains(location,New)], ReadSchema: struct<_c0:string,geo:string,text:string,user:string,location:string,entities:string,sentiment:st...




In [30]:
tdf.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [31]:
twitter_df

DataFrame[_c0: string, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [32]:
twitter_df.first()

Row(_c0='0', geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [33]:
twitter_df.take(5)

[Row(_c0='0', geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0='1', geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0='2', geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0='3', geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz',

In [34]:
twitter_df.select("text").show()

+--------------------+
|                text|
+--------------------+
|What is God sayin...|
|"BREAKING: ""this...|
| #Coronavirus tes...|
| Get ready for ma...|
| The #coronavirus...|
| COVID-19 update ...|
| It’s painful to ...|
| 📽️Friends, I wi...|
| Questions about ...|
|How they’re deali...|
| BREAKING: Democr...|
| “If we close dow...|
| I pity the poor ...|
| We’re the heck w...|
| I don't think sh...|
| Well written, so...|
| 1/2 CDC Director...|
| In all seriousne...|
| Wash your hands....|
| #CoronaVirusCana...|
+--------------------+
only showing top 20 rows



In [35]:
twitter_df.select("text","user").show()

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|What is God sayin...|          petodinice|
|"BREAKING: ""this...| but i took the t...|
| #Coronavirus tes...|              cek422|
| Get ready for ma...|        InfectiousDz|
| The #coronavirus...|          vic_gibson|
| COVID-19 update ...|      StewartNgilana|
| It’s painful to ...|           BWheatnyc|
| 📽️Friends, I wi...|             LorseaR|
| Questions about ...|         straightj23|
|How they’re deali...|       _______coolio|
| BREAKING: Democr...|      champagneaylin|
| “If we close dow...|       YorkLawLondon|
| I pity the poor ...|      BeesonMargaret|
| We’re the heck w...|      harrytiffanyiv|
| I don't think sh...|         grammyheath|
| Well written, so...|      barbara_ellena|
| 1/2 CDC Director...|               fatal|
| In all seriousne...|          pwjkmiller|
| Wash your hands....|        Mrrandy123RP|
| #CoronaVirusCana...|           

In [36]:
twitter_df.rdd.map(lambda line:line.text.split(" ")).take(5)

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  '']]

In [37]:
twitter_df.rdd.flatMap(lambda line:line.text.split(" ")).take(100)

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free',
 'to',
 'the',
 'public',
 'if',
 'we',
 'are',
 'going',
 'to',
 'understand',
 'the',
 'scope',
 'of',
 'this',
 'crisis.',
 'Anything',
 'le…',
 '',
 'Get',
 'ready',
 'for',
 'mass',
 'event',
 'crowd',
 'cancellations',
 'across',
 'the',
 'World',
 'starting',
 'this',
 'weekend:',
 'cricket',
 'in',
 '#Australia',
 'in',
 'empty',
 'st…',
 '',
 'The',
 '#coronavirus',
 'pandemic',
 'is',
 'revealing',
 'just',
 'how',
 'closely',
 'we',
 'are',
 'all',
 'bound',
 'together...[A',
 'thread]',
 '',
 '',
 'COVID-19',
 'update',
 'as',
 'of',
 'this',
 'morning:1.',
 'Death',
 'toll',
 'in',
 'Italy',
 'passes',
 '1,0002.',
 "Arsenal's",
 'head',
 'coach',
 'Arteta',
 'tests',
 'positive3.',
 'US…',
 '',
 'It’s',
 'painful',
 'to',
 'say,',
 'but',
 'as',
 'an']

In [38]:
corona_df.show(10)

+---+----------------+---------+--------+---------+----------+---------+-----+---------+----------------+----+
|_c0|           State|  Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|City|
+---+----------------+---------+--------+---------+----------+---------+-----+---------+----------------+----+
|  0|            null| Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|         Bangkok|null|
|  1|            null|    Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|         Hiraide|null|
|  2|            null|Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|       Singapore|null|
|  3|            null|    Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|       Kathmandu|null|
|  4|            null| Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|         Sarawak|null|
|  5|British Columbia|   Canada| 49.2827|-123.1207|2020-01-22|        0|    0|        0|British Columbia|null|
|

In [39]:
corona_df.filter("country='US'").show()

+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princess|     US|           35.4437|  139.638|2020-01-22|        0|   

In [40]:
corona_df.filter("country='US'").sort(col("date"), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [41]:
corona_df.filter("country='US'").orderBy(col("date"), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [42]:
corona_df.filter("Country='US'").sort([col("Date"),col("Confirmed")],ascending=False).show()

+-----+--------------------+-------+-------+---------+----------+---------+-----+---------+--------------------+--------------------+
|  _c0|               State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|                City|
+-----+--------------------+-------+-------+---------+----------+---------+-----+---------+--------------------+--------------------+
|27808|            Arkansas|     US|34.9697| -92.3731|2020-03-20|       96|    0|        0|            Arkansas|            Arkansas|
|27773|          New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|          New Jersey|          New Jersey|
|27785|             Indiana|     US|39.8494| -86.2583|2020-03-20|       86|    2|        0|             Indiana|             Indiana|
|27765|            New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|            New York|            New York|
|28080|             Alabama|     US|32.3182| -86.9023|2020-03-

In [43]:
corona_df.dtypes

[('_c0', 'string'),
 ('State', 'string'),
 ('Country', 'string'),
 ('Lat', 'string'),
 ('Long', 'string'),
 ('Date', 'string'),
 ('Confirmed', 'string'),
 ('Death', 'string'),
 ('Recovered', 'string'),
 ('state_cleaned', 'string'),
 ('City', 'string')]

In [44]:
corona_df=corona_df.withColumn("_c0",corona_df['_c0'].cast('int')).withColumn("Confirmed",corona_df["Confirmed"].cast('int'))\
.withColumn("Death",corona_df['Death'].cast('int')).withColumn('Recovered',corona_df['Recovered'].cast('int'))

In [45]:

corona_df.dtypes

[('_c0', 'int'),
 ('State', 'string'),
 ('Country', 'string'),
 ('Lat', 'string'),
 ('Long', 'string'),
 ('Date', 'string'),
 ('Confirmed', 'int'),
 ('Death', 'int'),
 ('Recovered', 'int'),
 ('state_cleaned', 'string'),
 ('City', 'string')]

In [46]:
corona_df.filter("Country='US'").sort([col("Date"),col("Confirmed")],ascending=False).show()

+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20|      585|    5|        0|      Illinois|      Illinois|
|27772|       Florida|     US|27.7663| -81.6868|2020-03-20|      563|   

In [47]:
corona_df.filter("Confirmed>10000").sort(col('Confirmed')).show()

+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18|    12327|   28|      105|       Berlin|null|
|23866|  null|  Italy|   43.0|    12.0|2

In [48]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|      Date|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|     28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|      null|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575|   

In [49]:
corona_df.filter('Confirmed>10000').sort('Confirmed').show()

+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18|    12327|   28|      105|       Berlin|null|
|23866|  null|  Italy|   43.0|    12.0|2

In [53]:
corona_df.approxQuantile('Confirmed',[.25,.5,.75,.9,.95],0)

[0.0, 0.0, 0.0, 38.0, 177.0]

In [54]:
corona_df.filter('Confirmed>10000').approxQuantile('Confirmed',[.25,.5,.75,.9,.95],0)

[17660.0, 41035.0, 66907.0, 67786.0, 67799.0]

In [55]:
corona_df.filter('Confirmed>10000').approxQuantile('Confirmed',[.25,.5,.75,.9,.95],.9)

[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [56]:
corona_df.agg({'date':'max'}).collect()

[Row(max(date)='2020-03-20')]

In [57]:
corona_df.agg({'date':'max'})

DataFrame[max(date): string]

In [58]:
corona_df.agg({'date':'max'}).collect()

[Row(max(date)='2020-03-20')]

In [59]:
corona_df.agg({'Date':'max','Confirmed':'max'}).collect()

[Row(max(Confirmed)=67800, max(Date)='2020-03-20')]

In [60]:
max_date=corona_df.agg({'date':'max'})

In [61]:
max_date.show()

+----------+
| max(date)|
+----------+
|2020-03-20|
+----------+



In [62]:
import pyspark.sql.functions as F

In [64]:
corona_df.groupby('Country','state_cleaned').agg(F.max('Date')).show()

+--------------+--------------------+----------+
|       Country|       state_cleaned| max(Date)|
+--------------+--------------------+----------+
|      Cameroon|             Yaounde|2020-03-20|
|         China|             Qinghai|2020-03-20|
|        Cyprus|             Nicosia|2020-03-20|
|            US|            Michigan|2020-03-20|
|      Portugal|              Lisbon|2020-03-20|
|            US|            Colorado|2020-03-20|
|United Kingdom|      Cayman Islands|2020-03-20|
|         China|              Hainan|2020-03-20|
|            US|            Missouri|2020-03-20|
|     Australia|Australian Capita...|2020-03-20|
|            US|                Guam|2020-03-20|
|        France|             Reunion|2020-03-20|
|      Colombia|        Cundinamarca|2020-03-20|
|          Cuba|              Havana|2020-03-20|
|     Mauritius|          Port Louis|2020-03-20|
|       Ukraine|                Kiev|2020-03-20|
|         Benin|          Porto-Novo|2020-03-20|
|   Switzerland|    

In [66]:
corona_df.groupby('Country','state_cleaned').agg(F.max('Date')).orderBy('Country','state_cleaned').show()

+-------------------+--------------------+----------+
|            Country|       state_cleaned| max(Date)|
+-------------------+--------------------+----------+
|        Afghanistan|             Baghran|2020-03-20|
|            Albania|                E852|2020-03-20|
|            Algeria|            Timokten|2020-03-20|
|            Andorra|    Andorra la Vella|2020-03-20|
|             Angola|             Sautari|2020-03-20|
|Antigua and Barbuda|          St. John's|2020-03-20|
|          Argentina|        Buenos Aires|2020-03-20|
|            Armenia|             Yerevan|2020-03-20|
|          Australia|Australian Capita...|2020-03-20|
|          Australia|From Diamond Prin...|2020-03-20|
|          Australia|     New South Wales|2020-03-20|
|          Australia|  Northern Territory|2020-03-20|
|          Australia|          Queensland|2020-03-20|
|          Australia|     South Australia|2020-03-20|
|          Australia|            Tasmania|2020-03-20|
|          Australia|       

In [71]:
corona_df.groupby('Country').pivot('Date').agg(F.sum('Confirmed')).orderBy('Country').collect()

[Row(Country='Afghanistan', 2020-01-22=0, 2020-01-23=0, 2020-01-24=0, 2020-01-25=0, 2020-01-26=0, 2020-01-27=0, 2020-01-28=0, 2020-01-29=0, 2020-01-30=0, 2020-01-31=0, 2020-02-01=0, 2020-02-02=0, 2020-02-03=0, 2020-02-04=0, 2020-02-05=0, 2020-02-06=0, 2020-02-07=0, 2020-02-08=0, 2020-02-09=0, 2020-02-10=0, 2020-02-11=0, 2020-02-12=0, 2020-02-13=0, 2020-02-14=0, 2020-02-15=0, 2020-02-16=0, 2020-02-17=0, 2020-02-18=0, 2020-02-19=0, 2020-02-20=0, 2020-02-21=0, 2020-02-22=0, 2020-02-23=0, 2020-02-24=1, 2020-02-25=1, 2020-02-26=1, 2020-02-27=1, 2020-02-28=1, 2020-02-29=1, 2020-03-01=1, 2020-03-02=1, 2020-03-03=1, 2020-03-04=1, 2020-03-05=1, 2020-03-06=1, 2020-03-07=1, 2020-03-08=4, 2020-03-09=4, 2020-03-10=5, 2020-03-11=7, 2020-03-12=7, 2020-03-13=7, 2020-03-14=11, 2020-03-15=16, 2020-03-16=21, 2020-03-17=22, 2020-03-18=22, 2020-03-19=22, 2020-03-20=24),
 Row(Country='Albania', 2020-01-22=0, 2020-01-23=0, 2020-01-24=0, 2020-01-25=0, 2020-01-26=0, 2020-01-27=0, 2020-01-28=0, 2020-01-29=0, 20

In [72]:
corona_df.groupby('Country').pivot('Date').agg(F.sum('Confirmed')).orderBy('Country').show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|            Country|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|

In [76]:
corona_df.groupBy("Country").agg({'Confirmed':'sum','Recovered':'sum','Death':'sum'}).orderBy('sum(Confirmed)',ascending=False).show()

+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|       1570179|    109614|       3287028|
|         Italy|         30834|     23658|        326105|
|          Iran|         56432|      9883|        188884|
|  Korea, South|         10523|      1233|        155106|
|         Spain|          7093|      4198|        103290|
|       Germany|          1063|       217|         89015|
|        France|           342|      1623|         71858|
|            US|           249|      1135|         67002|
|   Switzerland|            96|       220|         25348|
|   Cruise Ship|          3544|       165|         24092|
|United Kingdom|           607|       584|         18816|
|   Netherlands|            14|       373|         16031|
|         Japan|          2279|       337|         14471|
|        Norway|            12|        32|         13891|
|        Swede